<a href="https://colab.research.google.com/github/Kwanikaze/vpandas/blob/master/MRF_VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import matplotlib.pyplot as plt
import numpy as np

In [12]:
# Load synthetic data of one hot encoded digits - Does data need to respect Gaussian distribution?
# A,B,C inputs are all 8 bits ()
df = pd.read_csv("data_8.csv")
print(df.shape)

(5056, 3)


In [33]:
#Create two datasets containing AB and BC
num_samples = 1000
sample1_df = df[['A','B']].sample(n=num_samples, random_state=2)
print(sample1_df.head())
print("\n")
sample2_df = df[['B','C']].sample(n=num_samples, random_state=3)
print(sample2_df.head())

      A  B
4333  7  6
2638  6  4
2254  4  4
3116  5  5
3998  6  6


      B  C
4616  7  6
2276  4  6
3448  5  4
4064  6  5
1204  2  3


In [34]:
# Function to create two datasets from global df that are one-hot encoded
def OHE_sample(sample_df, features_to_OHE: list):
  for feature in features_to_OHE:
    feature_OHE = pd.get_dummies(sample1_df[feature])
    sample_df = pd.concat([sample_df,feature_OHE],axis=1)
  sample_df.drop(features_to_OHE,axis=1,inplace=True)
  print(sample_df)
  return sample_df

In [35]:
#sample1_A_OHE = pd.get_dummies(sample1_df.A)
#sample1_B_OHE pd.get_dummies(sample1_df.B)
#sample1_OHE = pd.concat([sample1_A_OHE,sample1_B_OHE],axis=1)
sample1_OHE = OHE_sample(sample1_df,['A','B'])
sample2_OHE = OHE_sample(sample2_df,['B','C'])
print(sample1_OHE)


      0  1  2  3  4  5  6  7  0  1  2  3  4  5  6  7
4333  0  0  0  0  0  0  0  1  0  0  0  0  0  0  1  0
2638  0  0  0  0  0  0  1  0  0  0  0  0  1  0  0  0
2254  0  0  0  0  1  0  0  0  0  0  0  0  1  0  0  0
3116  0  0  0  0  0  1  0  0  0  0  0  0  0  1  0  0
3998  0  0  0  0  0  0  1  0  0  0  0  0  0  0  1  0
...  .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. ..
1857  0  1  0  0  0  0  0  0  0  0  0  1  0  0  0  0
3813  0  0  0  0  0  1  0  0  0  0  0  0  0  0  1  0
604   1  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0
621   1  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0
1322  0  1  0  0  0  0  0  0  0  0  1  0  0  0  0  0

[1000 rows x 16 columns]


KeyError: ignored

In [ ]:
#Convert pandas dataframes to list of lists of lists
# [ [[OHE A1],[OHE B1]], [[OHE A2],[OHE B2]], ...  ]
# First list is a sample of two variables, second sequence of 8 digits, One-hot encoded: 0 to 00000000
